In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from helpers import *
import yfinance as yf

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
import time

In [3]:
cursor = get_cursor()

In [4]:
from KalshiClientsBaseV2 import ExchangeClient

In [5]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [6]:
abbr_to_month = {v: k for k, v in month_to_abbr.items()}

In [7]:
from KalshiClientsBaseV2 import HttpError

In [8]:
ndx = yf.Ticker('^NDX').history(period='max')
spx = yf.Ticker('^SPX').history(period='max')

spx = spx.reset_index()[['Date', 'Open', 'Close']]
ndx = ndx.reset_index()[['Date', 'Open', 'Close']]

spx['Date'] = spx.apply(lambda x: date(x['Date'].year, x['Date'].month, x['Date'].day), axis=1)
ndx['Date'] = ndx.apply(lambda x: date(x['Date'].year, x['Date'].month, x['Date'].day), axis=1)

In [74]:
#Kalshi SP500 range market goes back to April 28, 2022, size of each range change from 50 to 25 after May 17, 2023
import math
dt = date(2022,4,28)
lb = None
ub = None
mp = None
spx_d = {'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
while dt < date.today():
    spx_slice = spx[spx['Date']==dt]
    if spx_slice.shape[0] == 0:
        dt += timedelta(days = 1)
        continue
    spx_open = float(spx_slice['Open'])
    spx_close = float(spx_slice['Close'])
    event_ticker = 'INX-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        try:
            event_ticker = event_ticker[:9] + 'D' + event_ticker[9:]
            event = exchange_client.get_event(event_ticker)
        except:
            dt += timedelta(days = 1)
            continue
    market_tickers = []
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        if 'or' in subtitle and ('below' in subtitle or 'lower' in subtitle):
            lb = -np.inf
            ub = float(split[0].replace(',',''))
        elif 'or' in subtitle and ('above' in subtitle or 'higher' in subtitle):
            ub = np.inf
            lb = float(split[0].replace(',',''))
        elif 'to' in subtitle:
            lb = float(split[0].replace(',',''))
            if dt < date(2023, 5, 17):
                ub = lb + 49.99
            else:
                ub = lb + 24.99
        else:
            continue
      
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            spx_d['bid'].append(bb)
            spx_d['ask'].append(ba)
            spx_d['lb'].append(lb)
            spx_d['ub'].append(ub)
            spx_d['date'].append(dt)        
            spx_d['open'].append(spx_open)
            spx_d['close'].append(spx_close)

    dt += timedelta(days=1)
spx_df = pd.DataFrame(data=spx_d)

In [75]:
#Kalshi NASDAQ100 range market goes back to May 1, 2022, size of each range change is 100import math
dt = date(2022,5,1)
lb = None
ub = None
mp = None
ndx_d = {'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
while dt < date.today():
    ndx_slice = ndx[ndx['Date']==dt]
    if ndx_slice.shape[0] == 0:
        dt += timedelta(days = 1)
        continue
    ndx_open = float(ndx_slice['Open'])
    ndx_close = float(ndx_slice['Close'])
    event_ticker = 'NASDAQ100-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        try:
            event_ticker = event_ticker[:9] + 'D' + event_ticker[9:]
            event = exchange_client.get_event(event_ticker)
        except:
            dt += timedelta(days = 1)
            continue

    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        if 'or' in subtitle and ('below' in subtitle or 'lower' in subtitle):
            lb = -np.inf
            ub = float(split[0].replace(',',''))
        elif 'or' in subtitle and ('above' in subtitle or 'higher' in subtitle):
            ub = np.inf
            lb = float(split[0].replace(',',''))
        elif 'to' in subtitle:
            lb = float(split[0].replace(',',''))
            ub = lb + 99.99
        else:
            continue
    
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            ndx_d['bid'].append(bb)
            ndx_d['ask'].append(ba)
            ndx_d['lb'].append(lb)
            ndx_d['ub'].append(ub)
            ndx_d['date'].append(dt)        
            ndx_d['open'].append(ndx_open)
            ndx_d['close'].append(ndx_close)

    dt += timedelta(days=1)
ndx_df = pd.DataFrame(data=ndx_d)

In [76]:
spx_df['in_interval'] = spx_df.apply(lambda x: 1 if (x['close']>= x['lb']) and x['close']<=x['ub'] else 0, axis=1)
ndx_df['in_interval'] = ndx_df.apply(lambda x: 1 if x['close'] >= x['lb'] and x['close'] <= x['ub'] else 0, axis=1)

In [80]:
len(spx_df['date'].unique())

33

In [78]:
len(ndx_df['date'].unique())

383

In [79]:
ndx_df.groupby(['date']).agg('sum')['in_interval'].describe()

count    383.000000
mean       0.994778
std        0.072168
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: in_interval, dtype: float64

In [81]:
ndx_df

,bid,ask,lb,ub,open,close,date,in_interval
0,6,9,-inf,12499.99,12864.830078,13075.849609,2022-05-02,0
1,4,7,12500.0,12599.99,12864.830078,13075.849609,2022-05-02,0
2,8,11,12600.0,12699.99,12864.830078,13075.849609,2022-05-02,0
3,13,16,12700.0,12799.99,12864.830078,13075.849609,2022-05-02,0
4,16,19,12800.0,12899.99,12864.830078,13075.849609,2022-05-02,0
...,...,...,...,...,...,...,...,...
3306,0,34,17900.0,17999.99,17970.210938,17933.330078,2024-02-26,1
3307,24,29,18000.0,18099.99,17970.210938,17933.330078,2024-02-26,0
3308,0,14,18100.0,18199.99,17970.210938,17933.330078,2024-02-26,0
3309,0,11,18200.0,18299.99,17970.210938,17933.330078,2024-02-26,0


In [87]:
ndx_df = ndx_df[ndx_df['date'].isin(list((ndx_df.groupby(['date']).agg('sum')['in_interval'] == 1).index))==True].reset_index(drop=True)


In [88]:
ndx_df

,bid,ask,lb,ub,open,close,date,in_interval
0,6,9,-inf,12499.99,12864.830078,13075.849609,2022-05-02,0
1,4,7,12500.0,12599.99,12864.830078,13075.849609,2022-05-02,0
2,8,11,12600.0,12699.99,12864.830078,13075.849609,2022-05-02,0
3,13,16,12700.0,12799.99,12864.830078,13075.849609,2022-05-02,0
4,16,19,12800.0,12899.99,12864.830078,13075.849609,2022-05-02,0
...,...,...,...,...,...,...,...,...
3306,0,34,17900.0,17999.99,17970.210938,17933.330078,2024-02-26,1
3307,24,29,18000.0,18099.99,17970.210938,17933.330078,2024-02-26,0
3308,0,14,18100.0,18199.99,17970.210938,17933.330078,2024-02-26,0
3309,0,11,18200.0,18299.99,17970.210938,17933.330078,2024-02-26,0


In [91]:
ndx_df['ask'].describe()

count    3311.000000
mean       25.080036
std        27.933757
min         1.000000
25%         5.000000
50%        16.000000
75%        31.000000
max       100.000000
Name: ask, dtype: float64

In [98]:
small_ask = ndx_df[ndx_df['ask']<=10].reset_index(drop=True)
small_ask

,bid,ask,lb,ub,open,close,date,in_interval
0,6,9,-inf,12499.99,12864.830078,13075.849609,2022-05-02,0
1,4,7,12500.0,12599.99,12864.830078,13075.849609,2022-05-02,0
2,6,9,13100.0,13199.99,12864.830078,13075.849609,2022-05-02,0
3,0,3,-inf,12399.99,13045.349609,13089.900391,2022-05-03,0
4,0,3,12400.0,12499.99,13045.349609,13089.900391,2022-05-03,0
...,...,...,...,...,...,...,...,...
1299,1,3,18300.0,18399.99,18052.589844,17937.609375,2024-02-23,0
1300,0,1,17500.0,17599.99,17970.210938,17933.330078,2024-02-26,0
1301,0,2,17600.0,17699.99,17970.210938,17933.330078,2024-02-26,0
1302,0,7,17700.0,17799.99,17970.210938,17933.330078,2024-02-26,0


In [99]:
small_ask['in_interval'].describe()['mean']*small_ask['in_interval'].describe()['count']

32.0

In [100]:
small_ask['in_interval'].describe()

count    1304.000000
mean        0.024540
std         0.154777
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: in_interval, dtype: float64

In [101]:
small_ask['ask'].describe()

count    1304.000000
mean        4.648773
std         2.732811
min         1.000000
25%         2.000000
50%         4.000000
75%         7.000000
max        10.000000
Name: ask, dtype: float64

In [103]:
for x in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]:
    slice_df = ndx_df[(ndx_df['ask']>=x) & (ndx_df['ask']<x+2)]
    count = slice_df.shape[0]
    mean = slice_df['in_interval'].describe()['mean']
    print('Min Ask: ' + str(x) + ', Max Ask: ' + str(x+2) + ', Sample Size: ' + str(count) + ', Expected in interval proportion: ' + str(((x+1)/100)) + ', actual in interval proportion: ' + str(mean))
    

Min Ask: 0, Max Ask: 2, Sample Size: 144, Expected in interval proportion: 0.01, actual in interval proportion: 0.006944444444444444
Min Ask: 2, Max Ask: 4, Sample Size: 441, Expected in interval proportion: 0.03, actual in interval proportion: 0.013605442176870748
Min Ask: 4, Max Ask: 6, Sample Size: 257, Expected in interval proportion: 0.05, actual in interval proportion: 0.027237354085603113
Min Ask: 6, Max Ask: 8, Sample Size: 218, Expected in interval proportion: 0.07, actual in interval proportion: 0.03669724770642202
Min Ask: 8, Max Ask: 10, Sample Size: 160, Expected in interval proportion: 0.09, actual in interval proportion: 0.04375
Min Ask: 10, Max Ask: 12, Sample Size: 160, Expected in interval proportion: 0.11, actual in interval proportion: 0.06875
Min Ask: 12, Max Ask: 14, Sample Size: 142, Expected in interval proportion: 0.13, actual in interval proportion: 0.1056338028169014
Min Ask: 14, Max Ask: 16, Sample Size: 128, Expected in interval proportion: 0.15, actual in 

In [104]:
'''
For 9:30 am ET prices, lifting the ask regardless of whether the ticker is 'in the money' or not seems to be 
a losing game. ie: asking prices are too high and remote events are not priced fairly either

'''

"\nFor 9:30 am ET prices, lifting the ask regardless of whether the ticker is 'in the money' or not seems to be \na losing game. ie: asking prices are too high and remote events are not priced fairly either\n\n"

In [105]:
#Kalshi SP500 range market goes back to April 28, 2022, size of each range change from 50 to 25 after May 17, 2023
dt = date(2022,4,28)
lb = None
ub = None
mp = None
spx_d = {'bid': [], 'ask': [], 'lb': [], 'ub': [], 'close': [], 'date': []}
while dt < date.today():
    spx_slice = spx[spx['Date']==dt]
    if spx_slice.shape[0] == 0:
        dt += timedelta(days = 1)
        continue
    spx_close = float(spx_slice['Close'])
    event_ticker = 'INX-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        try:
            event_ticker = event_ticker[:9] + 'D' + event_ticker[9:]
            event = exchange_client.get_event(event_ticker)
        except:
            dt += timedelta(days = 1)
            continue
    market_tickers = []
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 14, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        if 'or' in subtitle and ('below' in subtitle or 'lower' in subtitle):
            lb = -np.inf
            ub = float(split[0].replace(',',''))
        elif 'or' in subtitle and ('above' in subtitle or 'higher' in subtitle):
            ub = np.inf
            lb = float(split[0].replace(',',''))
        elif 'to' in subtitle:
            lb = float(split[0].replace(',',''))
            if dt < date(2023, 5, 17):
                ub = lb + 49.99
            else:
                ub = lb + 24.99
        else:
            continue
      
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 14:
            bb = info['yes_bid']
            ba = info['yes_ask']
            spx_d['bid'].append(bb)
            spx_d['ask'].append(ba)
            spx_d['lb'].append(lb)
            spx_d['ub'].append(ub)
            spx_d['date'].append(dt)
            spx_d['close'].append(spx_close)

    dt += timedelta(days=1)
spx_df = pd.DataFrame(data=spx_d)

In [106]:
#Kalshi NASDAQ100 range market goes back to May 1, 2022, size of each range change is 100import math
dt = date(2022,5,1)
lb = None
ub = None
mp = None
ndx_d = {'bid': [], 'ask': [], 'lb': [], 'ub': [], 'close': [], 'date': []}
while dt < date.today():
    ndx_slice = ndx[ndx['Date']==dt]
    if ndx_slice.shape[0] == 0:
        dt += timedelta(days = 1)
        continue
    ndx_close = float(ndx_slice['Close'])
    event_ticker = 'NASDAQ100-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        try:
            event_ticker = event_ticker[:9] + 'D' + event_ticker[9:]
            event = exchange_client.get_event(event_ticker)
        except:
            dt += timedelta(days = 1)
            continue

    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 14, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        if 'or' in subtitle and ('below' in subtitle or 'lower' in subtitle):
            lb = -np.inf
            ub = float(split[0].replace(',',''))
        elif 'or' in subtitle and ('above' in subtitle or 'higher' in subtitle):
            ub = np.inf
            lb = float(split[0].replace(',',''))
        elif 'to' in subtitle:
            lb = float(split[0].replace(',',''))
            ub = lb + 99.99
        else:
            continue
    
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 14:
            bb = info['yes_bid']
            ba = info['yes_ask']
            ndx_d['bid'].append(bb)
            ndx_d['ask'].append(ba)
            ndx_d['lb'].append(lb)
            ndx_d['ub'].append(ub)
            ndx_d['date'].append(dt)
            ndx_d['close'].append(ndx_close)

    dt += timedelta(days=1)
ndx_df = pd.DataFrame(data=ndx_d)

In [107]:
spx_df['in_interval'] = spx_df.apply(lambda x: 1 if (x['close']>= x['lb']) and x['close']<=x['ub'] else 0, axis=1)
ndx_df['in_interval'] = ndx_df.apply(lambda x: 1 if x['close'] >= x['lb'] and x['close'] <= x['ub'] else 0, axis=1)

In [108]:
ndx_df.groupby(['date']).agg('sum')['in_interval'].describe()

count    389.000000
mean       0.997429
std        0.087894
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: in_interval, dtype: float64

In [109]:
spx_df.groupby(['date']).agg('sum')['in_interval'].describe()

count    34.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: in_interval, dtype: float64

In [110]:
for x in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]:
    slice_df = ndx_df[(ndx_df['ask']>=x) & (ndx_df['ask']<x+2)]
    count = slice_df.shape[0]
    mean = slice_df['in_interval'].describe()['mean']
    print('Min Ask: ' + str(x) + ', Max Ask: ' + str(x+2) + ', Sample Size: ' + str(count) + ', Expected in interval proportion: ' + str(((x+1)/100)) + ', actual in interval proportion: ' + str(mean))
    

Min Ask: 0, Max Ask: 2, Sample Size: 385, Expected in interval proportion: 0.01, actual in interval proportion: 0.0025974025974025974
Min Ask: 2, Max Ask: 4, Sample Size: 480, Expected in interval proportion: 0.03, actual in interval proportion: 0.008333333333333333
Min Ask: 4, Max Ask: 6, Sample Size: 168, Expected in interval proportion: 0.05, actual in interval proportion: 0.017857142857142856
Min Ask: 6, Max Ask: 8, Sample Size: 103, Expected in interval proportion: 0.07, actual in interval proportion: 0.02912621359223301
Min Ask: 8, Max Ask: 10, Sample Size: 100, Expected in interval proportion: 0.09, actual in interval proportion: 0.06
Min Ask: 10, Max Ask: 12, Sample Size: 66, Expected in interval proportion: 0.11, actual in interval proportion: 0.07575757575757576
Min Ask: 12, Max Ask: 14, Sample Size: 51, Expected in interval proportion: 0.13, actual in interval proportion: 0.11764705882352941
Min Ask: 14, Max Ask: 16, Sample Size: 53, Expected in interval proportion: 0.15, ac